<a href="https://colab.research.google.com/github/dlkt101101/CS431/blob/main/A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CS431 Data Intensive Distributed Analytics
### Winter 2025 - Assignment 3
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** _Darren Lam Kin Teng_
* **ID:** _20977843_

---
#### Overview
For this assignment, you will be using Python and Spark to perform some graph analysis, using a graph of the Gnutella server network.   In this graph, each node represents a server, and each (directed) edge represents a connection between servers in Gnutella's peer-to-peer network.  The input file for this assignment, `p2p-Gnutella08-adj.txt`, represents the graph as an adjacency list.   Each server (node) is identified by a unique number, and each line in the file gives the adjacency list for a single server.
For example, this line:
> 91	243	1923	2194

gives the adjacency list for server `91`.   It indicates that there are edges from server `91` to servers `243`, `1923`, and `2194`.    According to the Stanford Network Analysis Project, which collected these data, [the graph includes 6301 servers and 20777 edges](http://snap.stanford.edu/data/p2p-Gnutella08.html).

Run the following block to install Spark and download the input file.

In [3]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!curl -Os https://student.cs.uwaterloo.ca/~cs451/spark/spark-3.4.3-bin-hadoop3.tgz
!tar xzf spark-3.4.3-bin-hadoop3.tgz
!pip install -q findspark
!curl -Os https://student.cs.uwaterloo.ca/~cs451/content/cs431/p2p-Gnutella08-adj.txt

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


### Or
Run the following block to download just the input file (for those running on their own machine)

In [4]:
!curl -Os https://student.cs.uwaterloo.ca/~cs451/content/cs431/p2p-Gnutella08-adj.txt

set the environment variables (needed on Colab, shouldn't be needed on your own machine - and if it is, these values are likely not correct!)

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.3-bin-hadoop3"


and then create a `SparkContext`:

In [6]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

spark_conf = SparkConf()\
  .setAppName("YourTest")\
  .setMaster("local[*]")

sc = SparkContext.getOrCreate(spark_conf)

#### Colab code to view the Spark UI
This box will definitely not work on your own machine, which is fine, on your own machine the box above will have given you a local link - ctrl+click should open it in most editor

In [5]:
from google.colab import output
output.serve_kernel_port_as_window(4040, path='/jobs/index.html')
# This will create a link below. You must click the link, do not copy & paste the URL as that's the "local" URL and won't work on your machine

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

---
## Important
### Watch Your Types
###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions.
###### This means: Do not change the type of value returned, do not add or remove parameters.
### No Top-Level Expressions
###### Do **NOT** write code outside of functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.  Better still, create new cells for your tests, then you do not need to remember to delete anything
### Dummy Values
###### The function prototypes all return a default value, labelled as a dummy.  This is so that the cell is runnable even if you do not do a question.  You should replace this value with the correct value.
### New Cells
###### The best way to test your code is to create a NEW code cell from which to call your functions.  We will **ONLY** run the code cells that **WE** provided.  So do not put any helper functions in these cells, unless the helper is **ONLY** for use with your tests.  Any helpers needed by your solutions themselves must go in the original code cell.  (All cells will be put into a single .py file, so you do not need to copy and paste a helper from one question to the next)

---
#### Question 1  (6/24 marks):

To get warmed up, you should first write Spark code to confirm or determine some basic properties of the Gnutella graph.  Write code in the provided functions that will return answers to the following questions, as specified in each function's documentation:
- How many nodes and edges are there in the graph?  (This should confirm the numbers given above.)
- How many nodes of each outdegree are there? That is, how many nodes have no outgoing edges, how many have one outgoing edge, how many have two outgoing edges, and so on?
- How many nodes of each indegree are there?

You should use Spark to answer these questions.   Do *not* load the entire graph into your Python driver program.

In [11]:
# Q1
def num_nodes_edges():
    """Returns a tuple (num_nodes, num_edges)"""
    #### Your code for Question 1.1 should go here
    graph = sc.textFile("p2p-Gnutella08-adj.txt").map(lambda x: x.strip().split('\t'))
    nodes = graph.count()
    edges = graph.map(lambda x: len(x[1:])).sum()
    return (nodes, edges)

def out_counts():
    """Returns a dictionary where the keys are the outdegrees, and the
    values are the number of nodes of the corresponding outdegree """
    #### Your code for Question 1.2 should go here
    graph = sc.textFile("p2p-Gnutella08-adj.txt").map(lambda x: x.strip().split('\t'))

    # the number of neighbours a source node has is the outdegree
    def processLine(line):
      id, *neighbours = line
      return (len(neighbours), 1)
    dictionary = graph.map(processLine).reduceByKey(lambda x,y: x+y).collectAsMap()
    return dictionary

def in_counts():
    """Returns a dictionary where the keys are the indegrees, and the
    values are the number of nodes of the corresponding indegree """
    #### Your code for Question 1.3 should go here
    graph = sc.textFile("p2p-Gnutella08-adj.txt").map(lambda x: x.strip().split('\t')).cache()

    # consider nodes that can be a source and target
    edges = graph.flatMap(lambda x: [(neighbour, 1) for neighbour in x[1:]])
    # consider nodes that are only sources
    source_node = graph.map(lambda x: (x[0], 0))
    # reduce
    allNodes = source_node.union(edges).reduceByKey(lambda x, y: x+y)
    inDegree_dict = (
        allNodes.map(lambda x: (x[1], 1))
        .reduceByKey(lambda x,y: x+y)
        .collectAsMap()
        )
    return inDegree_dict

---
Your main objective for this assignment is to perform *single source personalized page rank* over the Gnutella graph.  To get started, you should make sure that you have a clear understanding of ordinary (i.e., non-personalized) page rank.  Read the description of page rank in Section 5.3 of [the course textbook](https://lintool.github.io/MapReduceAlgorithms/index.html).   Personalized page rank is like ordinary page rank except:
- One node in the graph is designated as the *source* node. Personalized page rank is performed with respect to that source node.
- Personalized page rank is initialized by assigning all probability mass to the source node, and none to the other nodes. In contrast, ordinary page rank is initialized by giving all nodes the same probability mass.
- Whenever personalized page rank makes a random jump, it jumps back to the source node. In contrast, ordinary page rank may jump to any node.
- In personalized page rank, all probability mass lost dangling nodes is put back into the source nodes.  In ordinary page rank, lost mass is distributed evenly over all nodes.

#### Testing Tips

Look at the first few lines of the input file. Try using node 0 as the source node and run it for 1 iteration. The source node should have `jump_factor` for its rank, and its out neighbours should all have the same value - `(1 - jump_factor) / degree(node 0)`

Once that works, try doing the math for 2 iterations and then confirm that works, too. Make sure it's handling all cases correctly.

You might also consider making a smaller input file to test on so the math is easier to do.

#### Question 2  (10/24 marks):

Your task is to write a Spark program to perform personalized page rank over the Gnutella graph for a specified number of iterations, and of course a specific node. The function you will implement takes three input values:
- source node id (a non-negative integer)
- iteration count (a positive integer)
- random jump factor value (a float between 0 and 1) - This is 1-B as introduced in the lecture.

The function should perform personalized page rank, with respect to the specified source node, over the Gnutella graph, for the specified number of iterations, using Spark.
The output of your function should be a list of the 10 nodes with the highest personalized page rank with respect to the given source. For each of the 10 nodes, return the node's id and page rank value as a tuple. The list returned by the function should therefore look something like this: `[(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]`

In [25]:
# Q2
def personalized_page_rank(source_node_id, num_iterations, jump_factor):
    """Returns a list of the 10 nodes with the highest page rank value along with their value, as tuples
    [(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]"""
    # your solution to Question 2 here

    """
    Personal notes:
    * For personalized page rank, source node(s) must have a rank of 1/m where m is
      the number of source nodes. All other nodes have a rank of 0
    """
    def contributions_func(t):
      """
      Takes in a tuple t
      """
      node_id, *information = t
      incoming_edges, rank = information
      results = []
      for y in incoming_edges:
        y_contribution = allContributions.lookup(y)
        value = y_contribution[0] if y_contribution else 0.
        contribution = allContributions.lookup(node_id)
        results.append(contribution)
      return results

    graph = (
        sc.textFile("p2p-Gnutella08-adj.txt")
        .map(lambda x: x.strip().split('\t'))
        .map(lambda x: (x[0], [neighbour for neighbour in x[1:]]))
        .cache()
    ) # (id, [outgoing edges of id])

    # find all incoming edges (node_a, [incoming edges to node_a])
    incoming_edge = (
        graph.flatMap(lambda x: [(neighbour, x[0]) for neighbour in x[1][1:]])
        .groupByKey()
        .mapValues(list())
        .cache()
    )

    # Get N = total nodes
    N = graph.count()

    # initialize the rank of all nodes (id, rank)
    initRank = (
        graph.map(lambda line: (line[0], 1. if source_node_id == line[0] else 0.))
    )

    # calculate the rank of all nodes
    # need to calculate PR(y)/out_degree(y)
    for i in range(num_iterations):
      allContributions = (
          graph.join(initRank) # (id, ([outgoing edges], rank) )
          .mapValues(lambda x: x[1][1]/len(x[1][0])) #(id, updated rank)
          )

      # sum contributions for y -> x
      incomingContributions = (
          incoming_edge.join(allContributions) # (node_a, ([incoming edges to node_a], updated_rank) )
          .flatMap(contributions_func)
          .reduceByKey(lambda x,y: x+y)
      )

      # apply the page rank
      pageRank = (
          graph.map(lambda x: x[0], 0.)
          .join(incomingContributions)
          .mapValues(lambda x: (1-jump_factor)/N + jump_factor * x[1])
          .cache()
      )


    return pageRank.sortBy(lambda x: x[1], ascending=False).take(10)

In [26]:
personalized_page_rank(0, 1, 0.5)

Traceback (most recent call last):
  File "/content/spark-3.4.3-bin-hadoop3/python/pyspark/serializers.py", line 459, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/spark-3.4.3-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/content/spark-3.4.3-bin-hadoop3/python/pyspark/cloudpickle/cloudpickle_fast.py", line 632, in dump
    return Pickler.dump(self, obj)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/spark-3.4.3-bin-hadoop3/python/pyspark/rdd.py", line 371, in __getnewargs__
    raise RuntimeError(
RuntimeError: It appears that you are attempting to broadcast an RDD or reference an RDD from an action or transformation. RDD transformations and actions can only be invoked by the driver, not inside of other transformations; for example, rdd1.map(lambda x: rdd2.values.count() * x) is invalid because the values transformation and count action canno

PicklingError: Could not serialize object: RuntimeError: It appears that you are attempting to broadcast an RDD or reference an RDD from an action or transformation. RDD transformations and actions can only be invoked by the driver, not inside of other transformations; for example, rdd1.map(lambda x: rdd2.values.count() * x) is invalid because the values transformation and count action cannot be performed inside of the rdd1.map transformation. For more information, see SPARK-5063.

---
#### Question 3  (4/24 marks):

For the previous question, you implemented personalized page rank that ran for a specified number of iterations.  However, it is also common to write iterative algorithms that run until some specified termination condition is reached.
For example, for page rank, suppose the $p_i(x)$ represents the probability mass assigned to node $x$ after the $i$th iteration of the algorithm.  ($p_0(x)$ is the initial probability mass of node $x$.)   We define the change of $x$'s probability mass on the $i$th iteration as $\lvert p_i(x)-p_{i-1}(x) \rvert$.   Then, we can iterate personalized page rank until the maximum (over all nodes) change is less than a specified threshold, i.e, until all nodes' page ranks have converged.

For this question, modify your personalized page rank implementation from Question 2 so that it iterates until the
maximum node change is less than $\frac{0.5}{N}$, where $N$ represents the number of nodes in the graph.
This version of the function should take only two inputs: the source node id and the random jump factor.

In [ ]:
# Q3

def personalized_page_rank_stopping_criterion(source_node_id, jump_factor):
    """Returns a list of the 10 nodes with the highest page rank value along with their value, as tuples
    [(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]"""
    # your solution to Question 3 here
    return [] # dummy value



---
#### Question 4  (4/24 marks):

Spark provides the ability to *cache* RDDs.   This is useful when an RDD will be used more than once.   Instead of computing the same RDD multiple times, it can be computed once, cached, and then re-used from the cache.   Read about caching in the Spark textbook, or in the [persistence section of the Spark RDD programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence).   Caching can be particularly effective for iterative Spark applications, like those you are writing for this assignment.

For this question, go back to the code that you wrote to answer Question 3, and add caching.   (Caching will not affect the functionality of your code, i.e., what it computes.   It should only affect performance.)   Don't worry about different persistence levels.   Just use `cache()`, which caches RDDs at the default persistence level.

In addition to adding `cache()` calls in your code, use the text cell below to briefly explain how you decided which RDDs to cache.

If you were not able to finish Question 3, add caching annotations to your solution for Question 2 instead.

IMPORTANT: Because the files are small, the OS will retain the intermediate shuffle files so you won't actually see any improvement in you running times! (This means you have to think about what to cache rather than trying various places to trial and error it).

---
#### Your answer to Question 4:

*Replace this with your `cache()` explanation from Question 4.*
I added cache to the initial RDD that I created since it is the "source" data and has been used to manipulate the data to something useable.



---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.